### Classification of ag plastics in Oxnard, CA

Sentinel-2 classification of agricultural plastic use in Oxnard, CA based on Yuanyuan Tian's summer 2023 project.

Converted from the GEE javascript to the GEE python api

In [ ]:
import numpy as np
import pandas as pd
import ee
import geemap

In [ ]:
# ee.Authenticate(auth_mode='localhost')

In [ ]:
# ee.Initialize()
ee.Initialize(project='ee-annalisertaylor')

In [ ]:
oxnard_bnd_shp = 'data/agplastics/Oxnard.shp'
oxnard_bnd = geemap.shp_to_ee(oxnard_bnd_shp)